In [ ]:
'''
    @Author: King
    @Date: 2019.03.19
    @Purpose: 中文文本分类实战（七）—— Adversarial LSTM模型
    @Introduction:  下面介绍
    @Datasets: ChnSentiCorp 中文情感分析数据集，其目标是判断一段话的情感态度。
    @Link : https://github.com/jiangxinyang227/textClassifier/tree/master/RCNN
    @Reference : https://www.cnblogs.com/jiangxinyang/p/10208290.html
'''

## 数据集

ChnSentiCorp 中文情感分析数据集，其目标是判断一段话的情感态度，总共有三个数据文件，在/data/ChnSentiCorp_cn_data目录下，包括ChnSentiCorp_htl_all.csv。在进行文本分类时需要有标签的数据（labeledTrainData），数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为/data/ChnSentiCorp_htl_all.csv/labeledTrain.csv。


## Adversarial LSTM模型结构

RCNN模型来源于论文Adversarial Training Methods For Semi-Supervised Text Classification {https://arxiv.org/abs/1605.07725}。其模型结构如下右图所示：

![avatar](../images/Adversarial_LSTM.png)

　上图中左边为正常的LSTM结构，右图为Adversarial LSTM结构，可以看出在输出时加上了噪声。

　　Adversarial LSTM的核心思想是通过对word Embedding上添加噪音生成对抗样本，将对抗样本以和原始样本 同样的形式喂给模型，得到一个Adversarial Loss，通过和原始样本的loss相加得到新的损失，通过优化该新 的损失来训练模型，作者认为这种方法能对word embedding加上正则化，避免过拟合。

In [1]:
import os
import csv
import time
import datetime
import random
import json
import threading

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
# 配置参数

class TrainingConfig(object):
    epoches = 5
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = 128  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    epsilon = 5
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/ChnSentiCorp_cn_data/labeledCharTrain.csv"
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [12]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self.indexFreqs = []  # 统计词空间中的词在出现在多少个review中
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath,encoding="utf-8")
        labels = df["label"].tolist()
        review = df["review_cut_word"].tolist()
        #print("review:{0}".format(review[0:1]))
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 得到逆词频
        self._getWordIndexFreq(vocab, reviews)
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _getWordIndexFreq(self, vocab, reviews):
        """
        统计词汇空间中各个词出现在多少个文本中
        """
        reviewDicts = [dict(zip(review, range(len(review)))) for review in reviews]
        indexFreqs = [0] * len(vocab)
        for word in vocab:
            count = 0
            for review in reviewDicts:
                if word in review:
                    count += 1
            indexFreqs[self._wordToIndex[word]] = count
        
        self.indexFreqs = indexFreqs
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r" ,encoding='utf-8') as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [13]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (4612, 200)
train label shape: (4612, 1)
eval data shape: (1154, 200)


In [14]:
print(data.wordEmbedding.shape)

(4732, 200)


## 生成batch数据集

　　采用生成器的形式向模型输入batch数据集，（生成器可以避免将所有的数据加入到内存中）

In [15]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

## Adversarial LSTM模型

In [16]:
# 构建模型
class AdversarialLSTM(object):
    """
    Adversarial LSTM模型 用于文本分类
    """
    def __init__(self, config, wordEmbedding, indexFreqs):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.config = config
        
        # 根据词的频率计算权重
        indexFreqs[0], indexFreqs[1] = 20000, 10000
        weights = tf.cast(tf.reshape(indexFreqs / tf.reduce_sum(indexFreqs), [1, len(indexFreqs)]), dtype=tf.float32)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用词频计算新的词嵌入矩阵
            normWordEmbedding = self._normalize(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), weights)
            
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(normWordEmbedding, self.inputX)
            
         # 计算二元交叉熵损失 
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)
        
        with tf.name_scope("perturLoss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)
        
        self.loss = loss + perturLoss
            
    def _Bi_LSTMAttention(self, embeddedWords):
        """
        Bi-LSTM + Attention 的模型结构
        """
        
        config = self.config
        
        # 定义双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
           
            # 定义前向LSTM结构
            lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)
            # 定义反向LSTM结构
            lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)


            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                          self.embeddedWords, dtype=tf.float32,
                                                                          scope="bi-lstm")

        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self._attention(H)
            outputSize = config.model.hiddenSizes
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            
        return predictions
    
    def _attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output
    
    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        
        mean = tf.matmul(weights, wordEmbedding)
        print(mean)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)
        
        var = tf.matmul(weights, powWordEmbedding)
        print(var)
        stddev = tf.sqrt(1e-6 + var)
        
        return (wordEmbedding - mean) / stddev
    
    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, self.config.model.epsilon)
        return embedded + perturb
    
    def _scaleL2(self, x, norm_length):
        # shape(x) = (batch, num_timesteps, d)
        # Divide x by max(abs(x)) for a numerically stable L2 norm.
        # 2norm(x) = a * 2norm(x/a)
        # Scale over the full sequence, dims (1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keepdims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(
            tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keepdims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [19]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = 0
    try:
        auc = roc_auc_score(trueY, predY)
    except ValueError:
        pass
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [21]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
indexFreqs = data.indexFreqs

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = AdversarialLSTM(config, wordEmbedding, indexFreqs)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/AdversarialLSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/AdversarialLSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

Tensor("embedding/MatMul:0", shape=(1, 200), dtype=float32)
Tensor("embedding/MatMul_1:0", shape=(1, 200), dtype=float32)
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-

2019-03-19T11:31:29.669418, step: 40, loss: 2.7524514728122287, acc: 0.030355555555555556, auc: 0.0, precision: 1.0, recall: 0.030355555555555556
Saved model checkpoint to ../model/AdversarialLSTM/model/my-model-40

2019-03-19T11:31:34.772776, step: 41, loss: 0.5997612476348877, acc: 0.875, auc: 0.8797, precision: 1.0, recall: 0.0588
2019-03-19T11:31:39.185981, step: 42, loss: 0.6401287317276001, acc: 0.875, auc: 0.7801, precision: 0.5, recall: 0.0625
2019-03-19T11:31:43.667002, step: 43, loss: 0.4940829873085022, acc: 0.9141, auc: 0.8139, precision: 1.0, recall: 0.0833
2019-03-19T11:31:48.007400, step: 44, loss: 0.9688690900802612, acc: 0.7734, auc: 0.7779, precision: 1.0, recall: 0.0333
2019-03-19T11:31:52.314888, step: 45, loss: 0.8004611134529114, acc: 0.8125, auc: 0.7889, precision: 0.0, recall: 0.0
2019-03-19T11:31:56.621374, step: 46, loss: 0.6899350881576538, acc: 0.8438, auc: 0.7824, precision: 1.0, recall: 0.0476
2019-03-19T11:32:00.946813, step: 47, loss: 0.5371537208557129,

2019-03-19T11:39:24.544075, step: 98, loss: 0.422291100025177, acc: 0.9453, auc: 0.8785, precision: 1.0, recall: 0.5333
2019-03-19T11:39:29.781078, step: 99, loss: 0.4859163463115692, acc: 0.8906, auc: 0.8702, precision: 0.4, recall: 0.1538
2019-03-19T11:39:34.846537, step: 100, loss: 0.5195876359939575, acc: 0.8906, auc: 0.9051, precision: 0.6875, recall: 0.55

Evaluation:
2019-03-19T11:40:18.675383, step: 100, loss: 1.8533211019304063, acc: 0.5026222222222222, auc: 0.0, precision: 1.0, recall: 0.5026222222222222
Saved model checkpoint to ../model/AdversarialLSTM/model/my-model-100

2019-03-19T11:40:24.311318, step: 101, loss: 0.353743314743042, acc: 0.9141, auc: 0.9652, precision: 0.7, recall: 0.4667
2019-03-19T11:40:28.873123, step: 102, loss: 0.35567164421081543, acc: 0.9297, auc: 0.9445, precision: 0.875, recall: 0.4667
2019-03-19T11:40:33.349159, step: 103, loss: 0.46162140369415283, acc: 0.9062, auc: 0.8454, precision: 0.75, recall: 0.375
2019-03-19T11:40:37.997735, step: 104, l

2019-03-19T11:48:43.477165, step: 154, loss: 0.3724292516708374, acc: 0.9297, auc: 0.9571, precision: 0.8, recall: 0.6667
2019-03-19T11:48:51.398984, step: 155, loss: 0.6035870313644409, acc: 0.875, auc: 0.9112, precision: 1.0, recall: 0.2727
2019-03-19T11:48:59.277922, step: 156, loss: 0.543547511100769, acc: 0.875, auc: 0.9126, precision: 1.0, recall: 0.1111
2019-03-19T11:49:06.133598, step: 157, loss: 0.47186097502708435, acc: 0.8984, auc: 0.9014, precision: 0.7, recall: 0.4118
2019-03-19T11:49:14.106286, step: 158, loss: 0.46134689450263977, acc: 0.9062, auc: 0.9177, precision: 1.0, recall: 0.3333
2019-03-19T11:49:22.515808, step: 159, loss: 0.45328181982040405, acc: 0.9219, auc: 0.8984, precision: 0.875, recall: 0.4375
2019-03-19T11:49:30.572273, step: 160, loss: 0.46669691801071167, acc: 0.9062, auc: 0.9184, precision: 1.0, recall: 0.3684

Evaluation:
2019-03-19T11:50:42.792736, step: 160, loss: 1.6312008963690863, acc: 0.5894222222222223, auc: 0.0, precision: 1.0, recall: 0.5894

KeyboardInterrupt: 